<a href="https://colab.research.google.com/github/Trivng1e/D.E.R.M.A./blob/main/THE_FINAL_DERMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import PIL
import tensorflow as tf

import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

from tensorflow import keras
from keras import layers, models
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from sys import getsizeof

In [ ]:
img_width = 128
img_height = 128
batch_size = 128

In [ ]:
train_images = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/drive/MyDrive/DERMNET/Train/',
    labels='inferred',
    label_mode = "categorical", 
    color_mode='rgb',
    batch_size= batch_size,
    image_size=(img_height,img_width),
    shuffle=True,
    seed=123,
)

test_images = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/drive/MyDrive/DERMNET/Test/',
    labels="inferred",
    label_mode = "categorical", 
    color_mode="rgb",
    batch_size=batch_size,
    image_size=(img_height,img_width),
    shuffle=True,
    seed=123,
)

Found 2574 files belonging to 3 classes.
Found 744 files belonging to 3 classes.


In [ ]:
data_augmentation = keras.Sequential(
  [
    layers.RandomFlip("horizontal",
                      input_shape=(img_height,
                                  img_width,
                                  3)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
  ] 
)

In [ ]:
model = models.Sequential()
model.add(keras.Input(shape = (128,128, 3)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(3))

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 30, 30, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 28, 28, 64)        36928     
                                                                 
 flatten (Flatten)           (None, 50176)            

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
history = model.fit(
    train_images, 
    epochs = 10, 
    batch_size = 128, 
    validation_data = test_images
    )

Epoch 1/10
21/21 [==============================] - 429s 15s/step - loss: 123.0779 - accuracy: 0.4099 - val_loss: 0.9721 - val_accuracy: 0.4839
Epoch 2/10
21/21 [==============================] - 103s 5s/step - loss: 0.8857 - accuracy: 0.5660 - val_loss: 0.9129 - val_accuracy: 0.5941
Epoch 3/10
21/21 [==============================] - 101s 5s/step - loss: 0.7486 - accuracy: 0.6721 - val_loss: 1.0295 - val_accuracy: 0.6156
Epoch 4/10
21/21 [==============================] - 98s 5s/step - loss: 0.6605 - accuracy: 0.7253 - val_loss: 0.8643 - val_accuracy: 0.6559
Epoch 5/10
21/21 [==============================] - 100s 5s/step - loss: 0.5413 - accuracy: 0.7723 - val_loss: 0.9401 - val_accuracy: 0.6317
Epoch 6/10
21/21 [==============================] - 102s 5s/step - loss: 0.4570 - accuracy: 0.8166 - val_loss: 1.0042 - val_accuracy: 0.6492
Epoch 7/10
21/21 [==============================] - 110s 5s/step - loss: 0.4069 - accuracy: 0.8318 - val_loss: 0.9632 - val_accuracy: 0.6868
Epoch 8/10


In [ ]:
test_path = ("/content/drive/MyDrive/Model/test1.jpg")

class_names = train_images.class_names
print(class_names)

img = tf.keras.utils.load_img(
    test_path, target_size=(img_height, img_width)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

['Acne ', 'Atopic Dermatitis ', 'Eczema ']
1/1 [==============================] - 0s 181ms/step
This image most likely belongs to Acne  with a 75.50 percent confidence.


In [ ]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

tflite_model_name= "derma_final_model2"
# Save the model.
open(tflite_model_name, "wb").write(tflite_model)

3277112